In [19]:
import csv
import pandas as pd
import ipywidgets as widgets

In [13]:
def read_csv(file, read_empty_lines=True, formats={}, encoding='utf-8', delimiter=';', quotechar='"', verbose = True):
    with open (file=file, mode='r', encoding=encoding) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=delimiter, quotechar=quotechar)
        header = next(csv_reader)
        for i, column in enumerate(header):
            header[i] = column.strip().replace('\ufeff', '')
        n_columns = len(header)
        n_skipped_rows = 0
        skipped_row = []
        rows = []
        is_individual_format = False
        all_format = 0
        if 'all' in formats.keys():
            if formats['all'] == 'str':
                is_individual_format = False
                all_format = 1
        for i, row in enumerate(csv_reader):
            #print(', '.join(row))
            assert len(row) == n_columns, 'Počet sloupců je divný'
            for j, column in enumerate(header):
                if not is_individual_format and all_format == 0:
                    try:
                        cislo = int(row[j].strip().replace(' ', '').replace(',', '.'))
                    except Exception as e:
                        cislo = row[j].strip()
                        try:
                            cislo = float(row[j].strip().replace(' ', '').replace(',', '.'))
                        except Exception as e:
                            cislo = row[j].strip()
                    if str(cislo) != row[j].strip().replace(' ', '').replace(',', '.'):
                        cislo = row[j].strip()
                    #print(str(cislo), row[j].strip().replace(' ', '').replace(',', '.'))
                else:
                    cislo = row[j].strip()
                if j == 0:    #radky
                    if isinstance(cislo, str) and cislo == '' and not read_empty_lines:
                        skipped_row = ['']
                    else:
                        skipped_row = []
                        rows.append([cislo])
                else:    #cisla
                    if len(skipped_row) == 0:
                        rows[-1].append(cislo)
                    else:
                        if cislo == '':
                            skipped_row.append('')
                            if j == n_columns - 1: #last empty collumn in skipped row
                                n_skipped_rows += 1
                        else:
                            #not an empty row
                            rows.append(skipped_row)
                            skipped_row = []
                            rows[-1].append(cislo)
    assert csv_reader.line_num - 1 - n_skipped_rows == len(rows), 'Počet řádků je divný'
    if verbose:
        print(f'Skipped {n_skipped_rows} empty rows!')
    return rows, header

In [14]:
file, delimiter = '_NACE_data/nace.csv', '\t'
data, columns = read_csv(file, read_empty_lines=False, delimiter=delimiter)

Skipped 0 empty rows!


In [15]:
data

[['A', '', '', '', 'SEKCE A - ZEMĚDĚLSTVÍ, LESNICTVÍ A\xa0RYBÁŘSTVÍ'],
 ['A',
  '01',
  '',
  '',
  'Rostlinná a živočišná výroba, myslivost a související činnosti'],
 ['A', '01', '01.1', '', 'Pěstování plodin jiných než trvalých'],
 ['A',
  '01',
  '01.1',
  '01.11',
  'Pěstování obilovin (kromě rýže), luštěnin a olejnatých semen'],
 ['A', '01', '01.1', '01.12', 'Pěstování rýže'],
 ['A', '01', '01.1', '01.13', 'Pěstování zeleniny a melounů, kořenů a hlíz'],
 ['A', '01', '01.1', '01.14', 'Pěstování cukrové třtiny'],
 ['A', '01', '01.1', '01.15', 'Pěstování tabáku'],
 ['A', '01', '01.1', '01.16', 'Pěstování přadných rostlin'],
 ['A',
  '01',
  '01.1',
  '01.19',
  'Pěstování ostatních plodin jiných než trvalých'],
 ['A', '01', '01.2', '', 'Pěstování trvalých plodin'],
 ['A', '01', '01.2', '01.21', 'Pěstování vinných hroznů'],
 ['A', '01', '01.2', '01.22', 'Pěstování tropického a subtropického ovoce'],
 ['A', '01', '01.2', '01.23', 'Pěstování citrusových plodů'],
 ['A', '01', '01.2', '01

In [16]:
columns

['sekce', 'oddil', 'skupina', 'trida', 'nazev']

In [17]:
df = pd.DataFrame(data=data, columns=columns)
df

,sekce,oddil,skupina,trida,nazev
0,A,,,,"SEKCE A - ZEMĚDĚLSTVÍ, LESNICTVÍ A RYBÁŘSTVÍ"
1,A,01,,,"Rostlinná a živočišná výroba, myslivost a souv..."
2,A,01,01.1,,Pěstování plodin jiných než trvalých
3,A,01,01.1,01.11,"Pěstování obilovin (kromě rýže), luštěnin a ol..."
4,A,01,01.1,01.12,Pěstování rýže
...,...,...,...,...,...
1151,T,98,98.2,98.20,Činnosti domácností poskytujících blíže neurče...
1152,U,,,,SEKCE U - ČINNOSTI EXTERITORIÁLNÍCH ORGANIZACÍ...
1153,U,99,,,Činnosti exteritoriálních organizací a orgánů
1154,U,99,99.0,,Činnosti exteritoriálních organizací a orgánů


In [47]:
myList = df[df['oddil'] == ''][['sekce', 'nazev']].astype(str).agg(' '.join, axis=1).to_list()
myList

['A SEKCE A - ZEMĚDĚLSTVÍ, LESNICTVÍ A\xa0RYBÁŘSTVÍ',
 'B SEKCE B - TĚŽBA A DOBÝVÁNÍ',
 'C SEKCE C - ZPRACOVATELSKÝ PRŮMYSL',
 'D SEKCE D – VÝROBA A ROZVOD ELEKTŘINY, PLYNU, TEPLA A\xa0KLIMATIZOVANÉHO VZDUCHU',
 'E SEKCE E – ZÁSOBOVÁNÍ VODOU; ČINNOSTI SOUVISEJÍCÍ S\xa0ODPADNÍMI VODAMI, ODPADY A SANACEMI',
 'F SEKCE F - STAVEBNICTVÍ',
 'G SEKCE G - VELKOOBCHOD A MALOOBCHOD; OPRAVY A\xa0ÚDRŽBA MOTOROVÝCH VOZIDEL',
 'H SEKCE H - DOPRAVA A SKLADOVÁNÍ',
 'I SEKCE I - UBYTOVÁNÍ, STRAVOVÁNÍ A\xa0POHOSTINSTVÍ',
 'J SEKCE J - INFORMAČNÍ A KOMUNIKAČNÍ ČINNOSTI',
 'K SEKCE K - PENĚŽNICTVÍ A POJIŠŤOVNICTVÍ',
 'L SEKCE L - ČINNOSTI V\xa0OBLASTI NEMOVITOSTÍ',
 'M SEKCE M - PROFESNÍ, VĚDECKÉ A\xa0TECHNICKÉ ČINNOSTI',
 'N SEKCE N - ADMINISTRATIVNÍ A PODPŮRNÉ ČINNOSTI',
 'O SEKCE O - VEŘEJNÁ SPRÁVA A OBRANA; POVINNÉ SOCIÁLNÍ ZABEZPEČENÍ',
 'P SEKCE P – VZDĚLÁVÁNÍ',
 'Q SEKCE Q - ZDRAVOTNÍ A SOCIÁLNÍ PÉČE',
 'R SEKCE R - KULTURNÍ, ZÁBAVNÍ A\xa0REKREAČNÍ ČINNOSTI',
 'S SEKCE S - OSTATNÍ ČINNOSTI',
 'T SE

In [48]:
myList1 = df[(df['oddil'] != '') & (df['skupina'] == '')][['oddil', 'nazev']].astype(str).agg(' '.join, axis=1).to_list()
myList1

['01 Rostlinná a živočišná výroba, myslivost a související činnosti',
 '02 Lesnictví a těžba dřeva',
 '03 Rybolov a akvakultura',
 '05 Těžba a úprava černého a hnědého uhlí',
 '06 Těžba ropy a zemního plynu',
 '07 Těžba a úprava rud',
 '08 Ostatní těžba a dobývání',
 '09 Podpůrné činnosti při těžbě',
 '10 Výroba potravinářských výrobků',
 '11 Výroba nápojů',
 '12 Výroba tabákových výrobků',
 '13 Výroba textilií',
 '14 Výroba oděvů',
 '15 Výroba usní a souvisejících výrobků',
 '16 Zpracování dřeva, výroba dřevěných, korkových, proutěných a slaměných výrobků, kromě nábytku',
 '17 Výroba papíru a výrobků z papíru',
 '18 Tisk a rozmnožování nahraných nosičů',
 '19 Výroba koksu a rafinovaných ropných produktů',
 '20 Výroba chemických látek a chemických přípravků',
 '21 Výroba základních farmaceutických výrobků a\xa0farmaceutických přípravků',
 '22 Výroba pryžových a plastových výrobků',
 '23 Výroba ostatních nekovových minerálních výrobků',
 '24 Výroba základních kovů, hutní zpracování kovů

In [56]:
w = widgets.Dropdown(
    options=[tuple([i, i[0]]) for i in myList],
    #options=myList,
    #value=[tuple([j, i]) for i, j in enumerate(myList)][0],
    description='vyber:',
    disabled=False,
    layout={'width': 'max-content'}
)

x = widgets.Dropdown(
    options=[tuple([i, i[0]]) for i in myList1],
    #options=myList,
    #value=[tuple([j, i]) for i, j in enumerate(myList)][0],
    description='vyber:',
    disabled=False,
    layout={'width': 'max-content'}
)

#w = widgets.Checkbox(False, description='Moje')
result = None
show = None

def on_change(change):
    global result, show, x
    if change['type'] == 'change' and change['name'] == 'value':
        result = "changed from %s" % change['old'] + " to %s" % change['new']
        show = change['new']
        myList1 = df[(df['sekce'] == show) & (df['oddil'] != '') & (df['skupina'] == '')][['oddil', 'nazev']].astype(str).agg(' '.join, axis=1).to_list()
        x = widgets.Dropdown(
            options=[tuple([i, i[0]]) for i in myList1],
            #options=myList,
            #value=[tuple([j, i]) for i, j in enumerate(myList)][0],
            description='vyber:',
            disabled=False,
            layout={'width': 'max-content'})

w.observe(on_change)

display(w)

Dropdown(description='vyber:', layout=Layout(width='max-content'), options=(('A SEKCE A - ZEMĚDĚLSTVÍ, LESNICT…

In [62]:
display(x)

Dropdown(description='vyber:', layout=Layout(width='max-content'), options=(('10 Výroba potravinářských výrobk…